### start

In [2]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import pickle
import cloudpickle
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

# import local modules
# pyright: reportMissingImports=false
sys.path.append('..//src')

import sagemaker_wallets.wallet_modeler as wm
import sagemaker_wallets.workflow_orchestrator as wwo
from utils import ConfigError
import utils as u

# reload all modules
modules = [
    wm, wwo,
    u
]

# load all configs
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


u.export_code(
    code_directories=[
        # 'etls',
        'sagemaker_wallets',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[06/Jul/25 16:35:22] INFO [utils.export_code:1542] Consolidation complete. All files are saved in temp/consolidated_code.py
[06/Jul/25 16:35:22] MILESTONE [2564028623.<module>:108] Good morning, let's get to work


# Initial ETLs

# Code begins

#### initiate orchestrator

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


workflow_orch = wwo.WalletWorkflowOrchestrator(sage_wallets_config)

#### load and upload data

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffixes = [
    '231107',
    '240306'
]
workflow_orch.load_training_data(date_suffixes)

[06/Jul/25 16:26:03] INFO [workflow_orchestrator.load_training_data:89] Loading training data for 2 periods: ['231107', '240306']
[06/Jul/25 16:26:03] INFO [workflow_orchestrator.load_training_data:106] Training data loaded successfully: 8 splits, 16,000 total rows


In [17]:
# Upload and retrieve URIs
workflow_orch.upload_training_data(overwrite_existing=False)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[06/Jul/25 16:26:05] INFO [workflow_orchestrator.upload_training_data:144] Ready to upload 16 training data files across 2 date folders.
[06/Jul/25 16:26:05] INFO [workflow_orchestrator.upload_training_data:145] Target: s3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/[DATE]/
[06/Jul/25 16:26:07] INFO [workflow_orchestrator.upload_training_data:167] File training_data_processed/dda_879_sagemaker_v0_dev/231107/x_train.csv already exists, skipping upload
[06/Jul/25 16:26:08] INFO [workflow_orchestrator.upload_training_data:167] File training_data_processed/dda_879_sagemaker_v0_dev/231107/x_test.csv already exists, skipping upload
[06/Jul/25 16:26:08] INFO [workflow_orchestrator.upload_training_data:167] File training_data_processed/dda_879_sagemaker_v0_dev/231107/x_eval.csv already exists, skipping upload
[06/Jul/25 16:26:08] INFO [workflow_orchestrator.upload_training_data:167] File training_data_processed/dda_879_sagemaker_v0_dev/231107/x_val.csv already exist

### retrieve uris

In [18]:
date_suffixes = [
    '231107',
    '240306'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

{'231107': {'x_train': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_train.csv',
  'y_train': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_train.csv',
  'x_test': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_test.csv',
  'y_test': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_test.csv',
  'x_eval': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_eval.csv',
  'y_eval': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_eval.csv',
  'x_val': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/x_val.csv',
  'y_val': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/231107/y_val.csv'},
 '240306': {'x_train': 's3://wallet-training-data/training_data_processed/dda_879_sagemaker_v0_dev/240306/x_train.csv',
  'y_

### dda 872 devspace

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


modeler = wm.WalletModeler(sage_wallets_config, workflow_orch.training_data)|




Custom TB Handler failed, unregistering


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
    [... skipping hidden 1 frame]

Cell In[19], line 5
      2 sagewallets_config = yaml.safe_load(Path('../config/sagewallets_config.yaml').read_text(encoding='utf-8'))
----> 5 modeler = wm.WalletModeler(sagewallets_config, workflow_orch.training_data)

File ~/DreamsData/GitHub/sagemaker/notebooks/..//src/sagemaker_wallets/wallet_modeler.py:42, in WalletModeler.__init__(self, sagewallets_config, training_data)
     41 self.training_data = training_data
---> 42 self._validate_training_data()
     44 self.model = None

AttributeError: 'WalletModeler' object has no attribute '_validate_training_data'

During handling of the above exception, another exception occurred:

TypeError                                 Traceback (most recent call last)
File ~/DreamsData/GitHub/data-science/src/utils.py:1245, in notify_on_failure(shell, etype, valu

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell. │
│ py:3577 in run_code                                                                              │
│                                                                                                  │
│   3574 │   │   │   │   if async_:                                                                │
│   3575 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3576 │   │   │   │   else:                                                                     │
│ ❱ 3577 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3578 │   │   │   finally:                                                                      │
│   3579 │   │   │   │   # Reset our crash handler in place                                        │
│   3580 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 sagewallets_config = yaml.safe_load(Path('../config/sagewallets_config.yaml').read_text(     │
│   3                                                                                              │
│   4                                                                                              │
│ ❱ 5 modeler = wm.WalletModeler(sagewallets_config, workflow_orch.training_data)                  │
│   6                                                                                              │
│   7                                                                                              │
│   8                                                                                              │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/..//src/sagemaker_wallets/wallet_model │
│ er.py:42 in __init__                                                                             │
│                                                                                                  │
│    39 │   │                                                                                      │
│    40 │   │   # Store and validate training data                                                 │
│    41 │   │   self.training_data = training_data                                                 │
│ ❱  42 │   │   self._validate_training_data()                                                     │
│    43 │   │                                                                                      │
│    44 │   │   self.model = None                                                                  │
│    45                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'WalletModeler' object has no attribute '_validate_training_data'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell. │
│ py:2025 in wrapped                                                                               │
│                                                                                                  │
│   2022 │   │   │   │   handlers to crash IPython.                                                │
│   2023 │

In [8]:
modeler.train_model()

[23/Jun/25 18:34:38] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 modeler.train_model()                                                                        │
│   2                                                                                              │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/..//src/sagemaker_wallets/wallet_model │
│ er.py:61 in train_model                                                                          │
│                                                                                                  │
│    58 │   │   role = self.sagewallets_config['aws']['modeler_arn']                               │
│    59 │   │                                                                                      │
│    60 │   │   # Convert data to CSV format and upload to S3                                      │
│ ❱  61 │   │   train_uri = self._upload_data_to_s3('train', sagemaker_session)                    │
│    62 │   │   eval_uri = self._upload_data_to_s3('eval', sagemaker_session)                      │
│    63 │   │                                                                                      │
│    64 │   │   # Configure XGBoost estimator                                                      │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/..//src/sagemaker_wallets/wallet_model │
│ er.py:170 in _upload_data_to_s3                                                                  │
│                                                                                                  │
│   167 │   │   # Upload to S3                                                                     │
│   168 │   │   bucket_name = 'wallet-training-data'                                               │
│   169 │   │                                                                                      │
│ ❱ 170 │   │   s3_uri = sagemaker_session.upload_data(                                            │
│   171 │   │   │   path=temp_file_path,                                                           │
│   172 │   │   │   bucket=bucket_name,                                                            │
│   173 │   │   │   key_prefix=folder_name                                                         │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-packages/sagemaker/session.py:432 in    │
│ upload_data                                                                                      │
│                                                                                                  │
│    429 │   │   │   s3 = self.s3_resource                                                         │
│    430 │   │                                                                                     │
│    431 │   │   for local_path, s3_key in files:                                                  │
│ ❱  432 │   │   │   s3.Object(bucket, s3_key).upload_file(                                        │
│    433 │   │   │   │   local_path, Callback=callback, ExtraArgs=extra_args                       │
│    434 │   │   │   )                                                                             │
│    435                                                                                           │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/.venv/lib/python3.11/site-pa

In [15]:
workflow_orch.training_data['x_train'].shape

(412616, 62)